In [1]:
import os
from datetime import datetime
from functools import reduce
from pathlib import Path

import numpy as np
import pandas as pd
from sqlalchemy import select
from sqlalchemy.orm import Session

from nanoHUB.application import Application
from nanoHUB.rfm.model import LastUpdateRecord, TempUserDescriptors, UserDescriptors

application = Application.get_instance()

nanoHUB - Serving Students, Researchers & Instructors


In [2]:
CACHE_DIR = Path(Path(os.getenv("APP_DIR")), ".cache")
NANOHUB_CACHE = Path(CACHE_DIR, "nanohub")
NANOHUB_METRICS_CACHE = Path(CACHE_DIR, "nanohub_metrics")

In [3]:
df = pd.read_csv(Path(CACHE_DIR, "toolevents_rfm.csv"))

FileNotFoundError: [Errno 2] No such file or directory: '/home/saxenap/nanoHUB/.cache/toolevents_rfm.csv'

In [ ]:
display(df)

In [ ]:
dfc = df[(df.first_start_date >= "2005-11-12")]
display(dfc)